# Merge IT datasets

In [58]:
IT_DATASETS_PATHS = [
    "datasets/instruction_tuning/bactrian-x/bactrian-x_cs_itformat",
    "datasets/instruction_tuning/muri-it/murit-it_cs_itformat",
    "datasets/instruction_tuning/oasst/oasst2_cs_itformat",
    "datasets/instruction_tuning/share_gpt/ShareGPT_cs_itformat",
    "datasets/instruction_tuning/ujc_cas/questions_ujc_cas_cs_itformat",
    "datasets/instruction_tuning/ask_library/ask_library_cs_itformat",
]

In [59]:
ID2DATASET = {
    0: "bactrian-x",
    1: "muri-it",
    2: "oasst2",
    3: "share_gpt",
    4: "questions_ujc_cas",
    5: "ask_library",
}

In [60]:
from datasets import load_from_disk

loaded_datasets = []

for dataset_path in IT_DATASETS_PATHS:
    loaded_datasets.append(load_from_disk(dataset_path))

loaded_datasets

[Dataset({
     features: ['instruction', 'input', 'id', 'output', 'conversations'],
     num_rows: 67017
 }),
 DatasetDict({
     train: Dataset({
         features: ['input', 'output', 'dataset_name', 'subdataset_name', 'language', 'split', 'language_name', 'conversations'],
         num_rows: 15875
     })
     validation: Dataset({
         features: ['input', 'output', 'dataset_name', 'subdataset_name', 'language', 'split', 'language_name', 'conversations'],
         num_rows: 882
     })
     test: Dataset({
         features: ['input', 'output', 'dataset_name', 'subdataset_name', 'language', 'split', 'language_name', 'conversations'],
         num_rows: 881
     })
 }),
 Dataset({
     features: ['id', 'conversations'],
     num_rows: 3
 }),
 Dataset({
     features: ['id', 'conversations'],
     num_rows: 187
 }),
 Dataset({
     features: ['question', 'specific_question', 'keyword', 'answer', 'last_usage', 'variants', 'id', 'conversations'],
     num_rows: 12363
 }),
 Dataset(

In [34]:
loaded_datasets[5]

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor', 'diacritics_correct', 'conversations'],
    num_rows: 13374
})

In [66]:
loaded_datasets[6][0]

IndexError: list index out of range

In [69]:
from datasets import Value

final_datasets = []
for i, dataset in enumerate(loaded_datasets):
    if ID2DATASET[i] == "bactrian-x":
        tmp_dataset = dataset
        tmp_dataset = tmp_dataset.remove_columns(["instruction", "input", "output"])
        tmp_dataset = tmp_dataset.rename_column("id", "original_id")
        tmp_dataset = tmp_dataset.map(lambda x: {"origin" : f"bactrian-x-{x['original_id'].split("-")[0]}"})
    elif ID2DATASET[i] == "muri-it": 
        tmp_dataset = dataset

        def muri_origin(x):
            if x["subdataset_name"] is None:
                x["origin"] = f"muri-it-{x['dataset_name']}"
            else:
                x["origin"] = f"muri-it-{x['dataset_name']}-{x['subdataset_name']}"
            return x
        
        tmp_dataset = tmp_dataset.map(muri_origin)
        tmp_dataset = tmp_dataset.remove_columns(["input", "output", "language", "split", "language_name", "dataset_name", "subdataset_name"])
        tmp_dataset = tmp_dataset.map(lambda _: {"origin" : "murit-it", "original_id": "murit-it"})

    elif ID2DATASET[i] == "oasst2":
        tmp_dataset = dataset
        tmp_dataset = tmp_dataset.rename_column("id", "original_id")
        tmp_dataset = tmp_dataset.map(lambda _: {"origin" : "oasst2"})

    elif ID2DATASET[i] == "share_gpt":
        tmp_dataset = dataset
        tmp_dataset = tmp_dataset.rename_column("id", "original_id")
        tmp_dataset = tmp_dataset.map(lambda _: {"origin" : "share_gpt"})
    elif ID2DATASET[i] == "questions_ujc_cas":
        tmp_dataset = dataset
        tmp_dataset = tmp_dataset.rename_column("id", "original_id")
        #convert to str
        tmp_dataset = tmp_dataset.cast_column("original_id", Value("string"))
        tmp_dataset = tmp_dataset.map(lambda _: {"origin" : "questions_ujc_cas"})
        tmp_dataset = tmp_dataset.remove_columns(["question", "specific_question", "keyword", "answer", "last_usage", "variants"])
    elif ID2DATASET[i] == "ask_library":
        tmp_dataset = dataset
        tmp_dataset = tmp_dataset.rename_column("id", "original_id")
        tmp_dataset = tmp_dataset.map(lambda _: {"origin" : "ask_library"})
        #convert to str
        tmp_dataset= tmp_dataset.cast_column("original_id", Value("string"))
        tmp_dataset = tmp_dataset.remove_columns(["title", "question", "answer", "okres", "library", "category", "used_sources", "date", "url", "language", 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor', 'diacritics_correct'])

    final_datasets.append(tmp_dataset)

Casting the dataset:   0%|          | 0/13374 [00:00<?, ? examples/s]

In [70]:
final_datasets

[Dataset({
     features: ['original_id', 'conversations', 'origin'],
     num_rows: 67017
 }),
 DatasetDict({
     train: Dataset({
         features: ['conversations', 'origin', 'original_id'],
         num_rows: 15875
     })
     validation: Dataset({
         features: ['conversations', 'origin', 'original_id'],
         num_rows: 882
     })
     test: Dataset({
         features: ['conversations', 'origin', 'original_id'],
         num_rows: 881
     })
 }),
 Dataset({
     features: ['original_id', 'conversations', 'origin'],
     num_rows: 3
 }),
 Dataset({
     features: ['original_id', 'conversations', 'origin'],
     num_rows: 187
 }),
 Dataset({
     features: ['original_id', 'conversations', 'origin'],
     num_rows: 12363
 }),
 Dataset({
     features: ['original_id', 'conversations', 'origin'],
     num_rows: 13374
 })]

In [73]:
from datasets import DatasetDict, Dataset, concatenate_datasets

split_datasets = []

for dataset in final_datasets:
    if isinstance(dataset, Dataset):
        if dataset.num_rows < 1000:
            test_size = 0.2
        elif dataset.num_rows < 20000:
            test_size = 0.05
        else:
            test_size = 0.02
        train_test = dataset.train_test_split(test_size=test_size, shuffle=True, seed=42)
        train = train_test["train"]
        test = train_test["test"]
        train_validation = train.train_test_split(test_size=test_size, shuffle=True, seed=42)
        train = train_validation["train"]
        validation = train_validation["test"]
        dataset = DatasetDict({"train": train, "validation": validation, "test": test})
        split_datasets.append(dataset)
    else:
        split_datasets.append(dataset)

split_datasets

[DatasetDict({
     train: Dataset({
         features: ['original_id', 'conversations', 'origin'],
         num_rows: 64362
     })
     validation: Dataset({
         features: ['original_id', 'conversations', 'origin'],
         num_rows: 1314
     })
     test: Dataset({
         features: ['original_id', 'conversations', 'origin'],
         num_rows: 1341
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['conversations', 'origin', 'original_id'],
         num_rows: 15875
     })
     validation: Dataset({
         features: ['conversations', 'origin', 'original_id'],
         num_rows: 882
     })
     test: Dataset({
         features: ['conversations', 'origin', 'original_id'],
         num_rows: 881
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['original_id', 'conversations', 'origin'],
         num_rows: 1
     })
     validation: Dataset({
         features: ['original_id', 'conversations', 'origin'],
         num_rows: 1
     })
  

In [74]:
train_datasets = []
validation_datasets = []
test_datasets = []

for dataset in split_datasets:
    train_datasets.append(dataset["train"])
    validation_datasets.append(dataset["validation"])
    test_datasets.append(dataset["test"])


train = concatenate_datasets(train_datasets)
validation = concatenate_datasets(validation_datasets)
test = concatenate_datasets(test_datasets)

final_dataset = DatasetDict({"train": train, "validation": validation, "test": test})
final_dataset

DatasetDict({
    train: Dataset({
        features: ['original_id', 'conversations', 'origin'],
        num_rows: 103582
    })
    validation: Dataset({
        features: ['original_id', 'conversations', 'origin'],
        num_rows: 3451
    })
    test: Dataset({
        features: ['original_id', 'conversations', 'origin'],
        num_rows: 3549
    })
})

In [76]:
final_dataset.push_to_hub("ctu-aic/cs_instruction_tuning_collection", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/104 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ctu-aic/cs_instruction_tuning_collection/commit/5748371223caa14f68e42cea667e215d30083118', commit_message='Upload dataset', commit_description='', oid='5748371223caa14f68e42cea667e215d30083118', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ctu-aic/cs_instruction_tuning_collection', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ctu-aic/cs_instruction_tuning_collection'), pr_revision=None, pr_num=None)

In [77]:
final_dataset.save_to_disk("data/it_datasets/cs_instruction_tuning_collection")

Saving the dataset (0/1 shards):   0%|          | 0/103582 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3451 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3549 [00:00<?, ? examples/s]